In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
# Install Pyspark & Setup our Java Environment - Takes less than a minute

!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.0.1/spark-3.0.1-bin-hadoop3.2.tgz
!tar -xvf spark-3.0.1-bin-hadoop3.2.tgz
!pip install -q findspark

tar: spark-3.0.1-bin-hadoop3.2.tgz: Cannot open: No such file or directory
tar: Error is not recoverable: exiting now


In [3]:
## installing pyspark
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.3/281.3 MB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - \ done
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=4524273651170e81ede69bbcde0bac8dcc1750033b744f1c54f0d438897afd42
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [4]:
## installing pyarrow
!pip install pyarrow

In [5]:
## creating a spark session
import findspark
findspark.init()

from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").appName("Spark ML on Titanic dataset").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/08/02 09:25:26 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [6]:
# Reading the dataset
df_train = spark.read.csv('/kaggle/input/titanic/train.csv', header = True, inferSchema=True)
df_test = spark.read.csv('/kaggle/input/titanic/test.csv', header = True, inferSchema=True)

In [7]:
df_train.show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|       S|
|          6|       0|     3|    Moran, Mr. James|  male|null|    0|    0|      

In [8]:
type(df_train)

pyspark.sql.dataframe.DataFrame

In [9]:
df_train.schema

StructType([StructField('PassengerId', IntegerType(), True), StructField('Survived', IntegerType(), True), StructField('Pclass', IntegerType(), True), StructField('Name', StringType(), True), StructField('Sex', StringType(), True), StructField('Age', DoubleType(), True), StructField('SibSp', IntegerType(), True), StructField('Parch', IntegerType(), True), StructField('Ticket', StringType(), True), StructField('Fare', DoubleType(), True), StructField('Cabin', StringType(), True), StructField('Embarked', StringType(), True)])

In [10]:
df_train.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [11]:
df_train.dtypes

[('PassengerId', 'int'),
 ('Survived', 'int'),
 ('Pclass', 'int'),
 ('Name', 'string'),
 ('Sex', 'string'),
 ('Age', 'double'),
 ('SibSp', 'int'),
 ('Parch', 'int'),
 ('Ticket', 'string'),
 ('Fare', 'double'),
 ('Cabin', 'string'),
 ('Embarked', 'string')]

In [12]:
df_train.describe().show()

+-------+-----------------+-------------------+------------------+--------------------+------+------------------+------------------+-------------------+------------------+-----------------+-----+--------+
|summary|      PassengerId|           Survived|            Pclass|                Name|   Sex|               Age|             SibSp|              Parch|            Ticket|             Fare|Cabin|Embarked|
+-------+-----------------+-------------------+------------------+--------------------+------+------------------+------------------+-------------------+------------------+-----------------+-----+--------+
|  count|              891|                891|               891|                 891|   891|               714|               891|                891|               891|              891|  204|     889|
|   mean|            446.0| 0.3838383838383838| 2.308641975308642|                null|  null| 29.69911764705882|0.5230078563411896|0.38159371492704824|260318.54916792738| 32.20420

In [13]:
df_train.select("Survived","Pclass","Embarked").show()

+--------+------+--------+
|Survived|Pclass|Embarked|
+--------+------+--------+
|       0|     3|       S|
|       1|     1|       C|
|       1|     3|       S|
|       1|     1|       S|
|       0|     3|       S|
|       0|     3|       Q|
|       0|     1|       S|
|       0|     3|       S|
|       1|     3|       S|
|       1|     2|       C|
|       1|     3|       S|
|       1|     1|       S|
|       0|     3|       S|
|       0|     3|       S|
|       0|     3|       S|
|       1|     2|       S|
|       0|     3|       Q|
|       1|     2|       S|
|       0|     3|       S|
|       1|     3|       C|
+--------+------+--------+
only showing top 20 rows



**EDA**

In [14]:
df_train.groupBy("Survived").count().show()

+--------+-----+
|Survived|count|
+--------+-----+
|       1|  342|
|       0|  549|
+--------+-----+



342 passengers survived and 549 did not survive

In [15]:
df_train.groupBy("Sex","Survived").count().show()

+------+--------+-----+
|   Sex|Survived|count|
+------+--------+-----+
|  male|       0|  468|
|female|       1|  233|
|female|       0|   81|
|  male|       1|  109|
+------+--------+-----+



out of 342 survived passengers, 233 are female and rest male; out of 549 deaths, 81 are female and rest male.

In [16]:
df_train.groupBy("Pclass","Survived").count().show()

+------+--------+-----+
|Pclass|Survived|count|
+------+--------+-----+
|     1|       0|   80|
|     3|       1|  119|
|     1|       1|  136|
|     2|       1|   87|
|     2|       0|   97|
|     3|       0|  372|
+------+--------+-----+



out of 342 survived, 136 belong to 1st class, 87 to 2nd class and 119 to 3rd class

out of 549 deaths, 80 belong to 1st class,97 to second class and 372 to 3rd class

**PREPROCESSING THE DATA**

In [17]:
# This function use to print feature with null values and null count 

from pyspark.sql.functions import mean,col,split,regexp_extract,when, lit

def null_value_count(df):
  null_columns_counts = []
  numRows = df.count()
  for k in df.columns:
    nullRows = df.where(col(k).isNull()).count()
    if(nullRows > 0):
      temp = k,nullRows
      null_columns_counts.append(temp)
  return(null_columns_counts)

In [18]:
# Calling function
null_columns_count_list = null_value_count(df_train)
null_columns_count_list

[('Age', 177), ('Cabin', 687), ('Embarked', 2)]

In [19]:
spark.createDataFrame(null_columns_count_list, ['Column_With_Null_Value', 'Null_Values_Count']).show()

+----------------------+-----------------+
|Column_With_Null_Value|Null_Values_Count|
+----------------------+-----------------+
|                   Age|              177|
|                 Cabin|              687|
|              Embarked|                2|
+----------------------+-----------------+



In [20]:
mean_age = df_train.select(mean('Age')).collect()[0][0]
print(mean_age)

29.69911764705882


In [21]:
df_train.groupBy("Embarked").count().show()

+--------+-----+
|Embarked|count|
+--------+-----+
|       Q|   77|
|    null|    2|
|       C|  168|
|       S|  644|
+--------+-----+



In [22]:
df_train = df_train.na.fill({"Embarked" : 'S'})

In [23]:
# Let's drop Cabin features as it has lots of null values
df_train = df_train.drop("Cabin")

In [24]:
df_train.show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05|       S|
|          6|       0|     3|    Moran, Mr. James|  male|null|    0|    0|          330877| 8.4583|       Q|
|          7|      

In [25]:
# Using the Regex ""[A-Za-z]+)." we extract the initials from the Name. 
# It looks for strings which lie between A-Z or a-z and followed by a .(dot).
df_train = df_train.withColumn("Initial",regexp_extract(col("Name"),"([A-Za-z]+)\.",1))

In [26]:
df_train.show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+--------+-------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Embarked|Initial|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+--------+-------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25|       S|     Mr|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|       C|    Mrs|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925|       S|   Miss|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1|       S|    Mrs|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05|       S|     Mr|
|          6|       0|     3|    Moran, Mr. James|  male|null|  

In [27]:
df_train.select("Initial").distinct().show()

+--------+
| Initial|
+--------+
|     Don|
|    Miss|
|Countess|
|     Col|
|     Rev|
|    Lady|
|  Master|
|     Mme|
|    Capt|
|      Mr|
|      Dr|
|     Mrs|
|     Sir|
|Jonkheer|
|    Mlle|
|   Major|
|      Ms|
+--------+



In [28]:
# We can see some misspelled initials like Mlle or Mme that stand for Miss. Let's replace them with Miss and same thing for other values
df_train = df_train.replace(['Mlle','Mme', 'Ms', 'Dr','Major','Dona','Lady','Countess','Jonkheer','Col','Rev','Capt','Sir','Don'],
               ['Miss','Miss','Miss','Mr','Mr',  'Mrs','Mrs', 'Mrs',  'Other',  'Other','Other','Mr','Mr','Mr'])

In [29]:
df_train.select("Initial").distinct().show()

+-------+
|Initial|
+-------+
|   Miss|
|  Other|
| Master|
|     Mr|
|    Mrs|
+-------+



In [30]:
df_train.groupby('Initial').avg('Age').collect()

[Row(Initial='Miss', avg(Age)=21.86),
 Row(Initial='Other', avg(Age)=45.888888888888886),
 Row(Initial='Master', avg(Age)=4.574166666666667),
 Row(Initial='Mr', avg(Age)=32.73960880195599),
 Row(Initial='Mrs', avg(Age)=35.981818181818184)]

In [31]:
df_train = df_train.withColumn("Age",when((df_train["Initial"] == "Miss") & (df_train["Age"].isNull()), 22).otherwise(df_train["Age"]))
df_train = df_train.withColumn("Age",when((df_train["Initial"] == "Other") & (df_train["Age"].isNull()), 44).otherwise(df_train["Age"]))
df_train = df_train.withColumn("Age",when((df_train["Initial"] == "Master") & (df_train["Age"].isNull()), 5).otherwise(df_train["Age"]))
df_train = df_train.withColumn("Age",when((df_train["Initial"] == "Mr") & (df_train["Age"].isNull()), 33).otherwise(df_train["Age"]))
df_train = df_train.withColumn("Age",when((df_train["Initial"] == "Mrs") & (df_train["Age"].isNull()), 37).otherwise(df_train["Age"]))

In [32]:
df_train.show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+--------+-------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Embarked|Initial|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+--------+-------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25|       S|     Mr|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|       C|    Mrs|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925|       S|   Miss|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1|       S|    Mrs|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05|       S|     Mr|
|          6|       0|     3|    Moran, Mr. James|  male|33.0|  

Creating a new feature called "Family_size",which is the summation of Parch(parents/children) and SibSp(siblings/spouses) and "Alone" when family size=0  and analyse it. 
This enables us to check if survival rate has anything to do with family size of the passengers

In [33]:
df_train = df_train.withColumn("Family_Size",col('SibSp')+col('Parch'))
df_train.groupBy("Family_Size").count().show()

+-----------+-----+
|Family_Size|count|
+-----------+-----+
|          1|  161|
|          6|   12|
|          3|   29|
|          5|   22|
|          4|   15|
|          7|    6|
|         10|    7|
|          2|  102|
|          0|  537|
+-----------+-----+



In [34]:
# 'Alone'=1 means person was alone
df_train = df_train.withColumn('Alone',lit(0))
df_train = df_train.withColumn("Alone",when(df_train["Family_Size"] == 0, 1).otherwise(df_train["Alone"]))

In [35]:
df_train.show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+--------+-------+-----------+-----+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Embarked|Initial|Family_Size|Alone|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+--------+-------+-----------+-----+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25|       S|     Mr|          1|    0|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|       C|    Mrs|          1|    0|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925|       S|   Miss|          0|    1|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1|       S|    Mrs|          1|    0|
|          5|       0|     3|Allen, Mr. Willia...|  mal

In [36]:
#converting Sex, Embarked & Initial columns from string to number using StringIndexer
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer
indexers = [StringIndexer(inputCol=column, outputCol=column+"_index").fit(df_train) for column in ["Sex","Embarked","Initial"]]
pipeline = Pipeline(stages=indexers)
df_train = pipeline.fit(df_train).transform(df_train)

In [37]:
df_train.show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+--------+-------+-----------+-----+---------+--------------+-------------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Embarked|Initial|Family_Size|Alone|Sex_index|Embarked_index|Initial_index|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+--------+-------+-----------+-----+---------+--------------+-------------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25|       S|     Mr|          1|    0|      0.0|           0.0|          0.0|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|       C|    Mrs|          1|    0|      1.0|           1.0|          2.0|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925|       S|   Miss|          0|  

In [38]:
df_train = df_train.drop("PassengerId","Name","Ticket","Cabin","Embarked","Sex","Initial")

In [39]:
df_train.show()

+--------+------+----+-----+-----+-------+-----------+-----+---------+--------------+-------------+
|Survived|Pclass| Age|SibSp|Parch|   Fare|Family_Size|Alone|Sex_index|Embarked_index|Initial_index|
+--------+------+----+-----+-----+-------+-----------+-----+---------+--------------+-------------+
|       0|     3|22.0|    1|    0|   7.25|          1|    0|      0.0|           0.0|          0.0|
|       1|     1|38.0|    1|    0|71.2833|          1|    0|      1.0|           1.0|          2.0|
|       1|     3|26.0|    0|    0|  7.925|          0|    1|      1.0|           0.0|          1.0|
|       1|     1|35.0|    1|    0|   53.1|          1|    0|      1.0|           0.0|          2.0|
|       0|     3|35.0|    0|    0|   8.05|          0|    1|      0.0|           0.0|          0.0|
|       0|     3|33.0|    0|    0| 8.4583|          0|    1|      0.0|           2.0|          0.0|
|       0|     1|54.0|    0|    0|51.8625|          0|    1|      0.0|           0.0|          0.0|


from output of describe function it is seen that minimum value of the column is 0.

In [40]:
df_train.filter(df_train["Fare"]==0).show()

+--------+------+----+-----+-----+----+-----------+-----+---------+--------------+-------------+
|Survived|Pclass| Age|SibSp|Parch|Fare|Family_Size|Alone|Sex_index|Embarked_index|Initial_index|
+--------+------+----+-----+-----+----+-----------+-----+---------+--------------+-------------+
|       0|     3|36.0|    0|    0| 0.0|          0|    1|      0.0|           0.0|          0.0|
|       0|     1|40.0|    0|    0| 0.0|          0|    1|      0.0|           0.0|          0.0|
|       1|     3|25.0|    0|    0| 0.0|          0|    1|      0.0|           0.0|          0.0|
|       0|     2|33.0|    0|    0| 0.0|          0|    1|      0.0|           0.0|          0.0|
|       0|     3|19.0|    0|    0| 0.0|          0|    1|      0.0|           0.0|          0.0|
|       0|     2|33.0|    0|    0| 0.0|          0|    1|      0.0|           0.0|          0.0|
|       0|     2|33.0|    0|    0| 0.0|          0|    1|      0.0|           0.0|          0.0|
|       0|     2|33.0|    0|  

In [41]:
# Get the average fares per class
df_train.groupby('Pclass').avg('Fare').collect()

[Row(Pclass=1, avg(Fare)=84.15468749999992),
 Row(Pclass=3, avg(Fare)=13.675550101832997),
 Row(Pclass=2, avg(Fare)=20.66218315217391)]

In [42]:
# Fill blank fare
df_train = df_train.withColumn("Fare",when((df_train["Pclass"] == 1) & (df_train["Fare"]==0), 84).otherwise(df_train["Fare"]))
df_train = df_train.withColumn("Fare",when((df_train["Pclass"] == 2) & (df_train["Fare"]==0), 20).otherwise(df_train["Fare"]))
df_train = df_train.withColumn("Fare",when((df_train["Pclass"] == 3) & (df_train["Fare"]==0), 13).otherwise(df_train["Fare"]))

In [43]:
df_train.show()

+--------+------+----+-----+-----+-------+-----------+-----+---------+--------------+-------------+
|Survived|Pclass| Age|SibSp|Parch|   Fare|Family_Size|Alone|Sex_index|Embarked_index|Initial_index|
+--------+------+----+-----+-----+-------+-----------+-----+---------+--------------+-------------+
|       0|     3|22.0|    1|    0|   7.25|          1|    0|      0.0|           0.0|          0.0|
|       1|     1|38.0|    1|    0|71.2833|          1|    0|      1.0|           1.0|          2.0|
|       1|     3|26.0|    0|    0|  7.925|          0|    1|      1.0|           0.0|          1.0|
|       1|     1|35.0|    1|    0|   53.1|          1|    0|      1.0|           0.0|          2.0|
|       0|     3|35.0|    0|    0|   8.05|          0|    1|      0.0|           0.0|          0.0|
|       0|     3|33.0|    0|    0| 8.4583|          0|    1|      0.0|           2.0|          0.0|
|       0|     1|54.0|    0|    0|51.8625|          0|    1|      0.0|           0.0|          0.0|


In [44]:
df_train.filter(df_train["Fare"]==0).show()

+--------+------+---+-----+-----+----+-----------+-----+---------+--------------+-------------+
|Survived|Pclass|Age|SibSp|Parch|Fare|Family_Size|Alone|Sex_index|Embarked_index|Initial_index|
+--------+------+---+-----+-----+----+-----------+-----+---------+--------------+-------------+
+--------+------+---+-----+-----+----+-----------+-----+---------+--------------+-------------+



In [45]:
#Putting all the features into a vector

from pyspark.ml.feature import VectorAssembler

feature = VectorAssembler(inputCols=df_train.columns[1:],outputCol="features")#leving the first column
feature_vector= feature.transform(df_train)

In [46]:
feature

VectorAssembler_14c67ca3b7a9

In [47]:
feature_vector.show()

+--------+------+----+-----+-----+-------+-----------+-----+---------+--------------+-------------+--------------------+
|Survived|Pclass| Age|SibSp|Parch|   Fare|Family_Size|Alone|Sex_index|Embarked_index|Initial_index|            features|
+--------+------+----+-----+-----+-------+-----------+-----+---------+--------------+-------------+--------------------+
|       0|     3|22.0|    1|    0|   7.25|          1|    0|      0.0|           0.0|          0.0|(10,[0,1,2,4,5],[...|
|       1|     1|38.0|    1|    0|71.2833|          1|    0|      1.0|           1.0|          2.0|[1.0,38.0,1.0,0.0...|
|       1|     3|26.0|    0|    0|  7.925|          0|    1|      1.0|           0.0|          1.0|[3.0,26.0,0.0,0.0...|
|       1|     1|35.0|    1|    0|   53.1|          1|    0|      1.0|           0.0|          2.0|[1.0,35.0,1.0,0.0...|
|       0|     3|35.0|    0|    0|   8.05|          0|    1|      0.0|           0.0|          0.0|(10,[0,1,4,6],[3....|
|       0|     3|33.0|    0|    

In [48]:
(trainingData, validData) = feature_vector.randomSplit([0.8, 0.2],seed = 11)

In [49]:
trainingData.show()

+--------+------+----+-----+-----+--------+-----------+-----+---------+--------------+-------------+--------------------+
|Survived|Pclass| Age|SibSp|Parch|    Fare|Family_Size|Alone|Sex_index|Embarked_index|Initial_index|            features|
+--------+------+----+-----+-----+--------+-----------+-----+---------+--------------+-------------+--------------------+
|       0|     1| 2.0|    1|    2|  151.55|          3|    0|      1.0|           0.0|          1.0|[1.0,2.0,1.0,2.0,...|
|       0|     1|18.0|    1|    0|   108.9|          1|    0|      0.0|           1.0|          0.0|[1.0,18.0,1.0,0.0...|
|       0|     1|19.0|    1|    0|    53.1|          1|    0|      0.0|           0.0|          0.0|(10,[0,1,2,4,5],[...|
|       0|     1|19.0|    3|    2|   263.0|          5|    0|      0.0|           0.0|          0.0|[1.0,19.0,3.0,2.0...|
|       0|     1|21.0|    0|    1| 77.2875|          1|    0|      0.0|           0.0|          0.0|(10,[0,1,3,4,5],[...|
|       0|     1|24.0|  

In [50]:
validData.show()

+--------+------+----+-----+-----+--------+-----------+-----+---------+--------------+-------------+--------------------+
|Survived|Pclass| Age|SibSp|Parch|    Fare|Family_Size|Alone|Sex_index|Embarked_index|Initial_index|            features|
+--------+------+----+-----+-----+--------+-----------+-----+---------+--------------+-------------+--------------------+
|       0|     1|22.0|    0|    0|135.6333|          0|    1|      0.0|           1.0|          0.0|(10,[0,1,4,6,8],[...|
|       0|     1|24.0|    0|    0|    79.2|          0|    1|      0.0|           1.0|          0.0|(10,[0,1,4,6,8],[...|
|       0|     1|29.0|    0|    0|    30.0|          0|    1|      0.0|           0.0|          0.0|(10,[0,1,4,6],[1....|
|       0|     1|29.0|    1|    0|    66.6|          1|    0|      0.0|           0.0|          0.0|(10,[0,1,2,4,5],[...|
|       0|     1|30.0|    0|    0|   27.75|          0|    1|      0.0|           1.0|          0.0|(10,[0,1,4,6,8],[...|
|       0|     1|31.0|  

**MODELLING**

In [51]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

lr = LogisticRegression(labelCol="Survived", featuresCol="features")

# Training our Model
lrModel = lr.fit(trainingData)
lr_prediction = lrModel.transform(validData)
lr_prediction.select("prediction", "Survived", "features").show()
evaluator = MulticlassClassificationEvaluator(labelCol="Survived", predictionCol="prediction", metricName="accuracy")

+----------+--------+--------------------+
|prediction|Survived|            features|
+----------+--------+--------------------+
|       1.0|       0|(10,[0,1,4,6,8],[...|
|       1.0|       0|(10,[0,1,4,6,8],[...|
|       0.0|       0|(10,[0,1,4,6],[1....|
|       0.0|       0|(10,[0,1,2,4,5],[...|
|       1.0|       0|(10,[0,1,4,6,8],[...|
|       0.0|       0|(10,[0,1,2,4,5],[...|
|       0.0|       0|(10,[0,1,4,6],[1....|
|       0.0|       0|(10,[0,1,4,6,8],[...|
|       0.0|       0|(10,[0,1,4,6],[1....|
|       0.0|       0|(10,[0,1,4,6],[1....|
|       0.0|       0|(10,[0,1,4,6],[1....|
|       0.0|       0|(10,[0,1,2,4,5],[...|
|       0.0|       0|(10,[0,1,3,4,5],[...|
|       0.0|       0|(10,[0,1,2,4,5],[...|
|       0.0|       0|(10,[0,1,4,6],[1....|
|       0.0|       0|[1.0,58.0,0.0,2.0...|
|       0.0|       0|(10,[0,1,4,6],[1....|
|       0.0|       0|(10,[0,1,4,6],[1....|
|       0.0|       0|(10,[0,1,4,6,8],[...|
|       0.0|       0|[2.0,19.0,1.0,1.0...|
+----------

In [52]:
lr_accuracy = evaluator.evaluate(lr_prediction)
print("Validation Accuracy of LogisticRegression is = %g"% (lr_accuracy))
print("Validation Error of LogisticRegression = %g " % (1.0 - lr_accuracy))

Validation Accuracy of LogisticRegression is = 0.781915
Validation Error of LogisticRegression = 0.218085 


In [53]:
df_test.show()

+-----------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|        892|     3|    Kelly, Mr. James|  male|34.5|    0|    0|          330911| 7.8292| null|       Q|
|        893|     3|Wilkes, Mrs. Jame...|female|47.0|    1|    0|          363272|    7.0| null|       S|
|        894|     2|Myles, Mr. Thomas...|  male|62.0|    0|    0|          240276| 9.6875| null|       Q|
|        895|     3|    Wirz, Mr. Albert|  male|27.0|    0|    0|          315154| 8.6625| null|       S|
|        896|     3|Hirvonen, Mrs. Al...|female|22.0|    1|    1|         3101298|12.2875| null|       S|
|        897|     3|Svensson, Mr. Joh...|  male|14.0|    0|    0|            7538|  9.225| null|       S|
|        898|     3|Connolly, Miss. Kate|femal

In [54]:
df_train.show()

+--------+------+----+-----+-----+-------+-----------+-----+---------+--------------+-------------+
|Survived|Pclass| Age|SibSp|Parch|   Fare|Family_Size|Alone|Sex_index|Embarked_index|Initial_index|
+--------+------+----+-----+-----+-------+-----------+-----+---------+--------------+-------------+
|       0|     3|22.0|    1|    0|   7.25|          1|    0|      0.0|           0.0|          0.0|
|       1|     1|38.0|    1|    0|71.2833|          1|    0|      1.0|           1.0|          2.0|
|       1|     3|26.0|    0|    0|  7.925|          0|    1|      1.0|           0.0|          1.0|
|       1|     1|35.0|    1|    0|   53.1|          1|    0|      1.0|           0.0|          2.0|
|       0|     3|35.0|    0|    0|   8.05|          0|    1|      0.0|           0.0|          0.0|
|       0|     3|33.0|    0|    0| 8.4583|          0|    1|      0.0|           2.0|          0.0|
|       0|     1|54.0|    0|    0|51.8625|          0|    1|      0.0|           0.0|          0.0|


**PREPROCESSING THE TEST DATA**

In [55]:
df_test = df_test.withColumn("Family_Size",col('SibSp')+col('Parch'))
df_test.groupBy("Family_Size").count().show()

+-----------+-----+
|Family_Size|count|
+-----------+-----+
|          1|   74|
|          6|    4|
|          3|   14|
|          5|    3|
|          4|    7|
|          7|    2|
|         10|    4|
|          2|   57|
|          0|  253|
+-----------+-----+



In [56]:
# 'Alone'=1 means person was alone
df_test = df_test.withColumn('Alone',lit(0))
df_test = df_test.withColumn("Alone",when(df_test["Family_Size"] == 0, 1).otherwise(df_test["Alone"]))

In [57]:
df_test.show()

+-----------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+-----------+-----+
|PassengerId|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|Family_Size|Alone|
+-----------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+-----------+-----+
|        892|     3|    Kelly, Mr. James|  male|34.5|    0|    0|          330911| 7.8292| null|       Q|          0|    1|
|        893|     3|Wilkes, Mrs. Jame...|female|47.0|    1|    0|          363272|    7.0| null|       S|          1|    0|
|        894|     2|Myles, Mr. Thomas...|  male|62.0|    0|    0|          240276| 9.6875| null|       Q|          0|    1|
|        895|     3|    Wirz, Mr. Albert|  male|27.0|    0|    0|          315154| 8.6625| null|       S|          0|    1|
|        896|     3|Hirvonen, Mrs. Al...|female|22.0|    1|    1|         3101298|12.2875| null|       S|          2|    0|
|       

In [58]:
df_test = df_test.withColumn("Initial",regexp_extract(col("Name"),"([A-Za-z]+)\.",1))

In [59]:
df_test.select("Initial").distinct().show()

+-------+
|Initial|
+-------+
|   Dona|
|   Miss|
|    Col|
|    Rev|
| Master|
|     Mr|
|     Dr|
|    Mrs|
|     Ms|
+-------+



In [60]:
df_test = df_test.replace(['Mlle','Mme', 'Ms', 'Dr','Major','Dona','Lady','Countess','Jonkheer','Col','Rev','Capt','Sir','Don'],
               ['Miss','Miss','Miss','Mr','Mr',  'Mrs','Mrs', 'Mrs',  'Other',  'Other','Other','Mr','Mr','Mr'])

In [61]:
df_test.select("Initial").distinct().show()

+-------+
|Initial|
+-------+
|   Miss|
|  Other|
| Master|
|     Mr|
|    Mrs|
+-------+



In [62]:
df_test.groupby('Initial').avg('Age').collect()

[Row(Initial='Miss', avg(Age)=21.774843750000002),
 Row(Initial='Other', avg(Age)=42.75),
 Row(Initial='Master', avg(Age)=7.406470588235294),
 Row(Initial='Mr', avg(Age)=32.11413043478261),
 Row(Initial='Mrs', avg(Age)=38.904761904761905)]

In [63]:
df_test = df_test.withColumn("Age",when((df_test["Initial"] == "Miss") & (df_test["Age"].isNull()), 21).otherwise(df_test["Age"]))
df_test = df_test.withColumn("Age",when((df_test["Initial"] == "Other") & (df_test["Age"].isNull()), 42).otherwise(df_test["Age"]))
df_test = df_test.withColumn("Age",when((df_test["Initial"] == "Master") & (df_test["Age"].isNull()), 7).otherwise(df_test["Age"]))
df_test = df_test.withColumn("Age",when((df_test["Initial"] == "Mr") & (df_test["Age"].isNull()), 32).otherwise(df_test["Age"]))
df_test = df_test.withColumn("Age",when((df_test["Initial"] == "Mrs") & (df_test["Age"].isNull()), 38).otherwise(df_test["Age"]))

In [64]:
#converting Sex, Embarked & Initial columns from string to number using StringIndexer
indexers = [StringIndexer(inputCol=column, outputCol=column+"_index").fit(df_test) for column in ["Sex","Embarked","Initial"]]
pipeline = Pipeline(stages=indexers)
df_test = pipeline.fit(df_test).transform(df_test)

In [65]:
df_test = df_test.drop("Name","Ticket","Cabin","Embarked","Sex","Initial")

In [66]:
df_test.show()

+-----------+------+----+-----+-----+-------+-----------+-----+---------+--------------+-------------+
|PassengerId|Pclass| Age|SibSp|Parch|   Fare|Family_Size|Alone|Sex_index|Embarked_index|Initial_index|
+-----------+------+----+-----+-----+-------+-----------+-----+---------+--------------+-------------+
|        892|     3|34.5|    0|    0| 7.8292|          0|    1|      0.0|           2.0|          0.0|
|        893|     3|47.0|    1|    0|    7.0|          1|    0|      1.0|           0.0|          2.0|
|        894|     2|62.0|    0|    0| 9.6875|          0|    1|      0.0|           2.0|          0.0|
|        895|     3|27.0|    0|    0| 8.6625|          0|    1|      0.0|           0.0|          0.0|
|        896|     3|22.0|    1|    1|12.2875|          2|    0|      1.0|           0.0|          2.0|
|        897|     3|14.0|    0|    0|  9.225|          0|    1|      0.0|           0.0|          0.0|
|        898|     3|30.0|    0|    0| 7.6292|          0|    1|      1.0|

In [67]:
df_test.filter(df_test["Fare"]==0).show()

+-----------+------+----+-----+-----+----+-----------+-----+---------+--------------+-------------+
|PassengerId|Pclass| Age|SibSp|Parch|Fare|Family_Size|Alone|Sex_index|Embarked_index|Initial_index|
+-----------+------+----+-----+-----+----+-----------+-----+---------+--------------+-------------+
|       1158|     1|32.0|    0|    0| 0.0|          0|    1|      0.0|           0.0|          0.0|
|       1264|     1|49.0|    0|    0| 0.0|          0|    1|      0.0|           0.0|          0.0|
+-----------+------+----+-----+-----+----+-----------+-----+---------+--------------+-------------+



In [68]:
# Get the average fares per class
df_test.groupby('Pclass').avg('Fare').collect()

[Row(Pclass=1, avg(Fare)=94.28029719626169),
 Row(Pclass=3, avg(Fare)=12.459677880184334),
 Row(Pclass=2, avg(Fare)=22.20210430107527)]

In [69]:
df_test = df_test.withColumn("Fare",when((df_test["Pclass"] == 1) & (df_test["Fare"]==0), 94).otherwise(df_test["Fare"]))
df_test = df_test.withColumn("Fare",when((df_test["Pclass"] == 2) & (df_test["Fare"]==0), 22).otherwise(df_test["Fare"]))
df_test = df_test.withColumn("Fare",when((df_test["Pclass"] == 3) & (df_test["Fare"]==0), 13).otherwise(df_test["Fare"]))

In [70]:
df_test.show()

+-----------+------+----+-----+-----+-------+-----------+-----+---------+--------------+-------------+
|PassengerId|Pclass| Age|SibSp|Parch|   Fare|Family_Size|Alone|Sex_index|Embarked_index|Initial_index|
+-----------+------+----+-----+-----+-------+-----------+-----+---------+--------------+-------------+
|        892|     3|34.5|    0|    0| 7.8292|          0|    1|      0.0|           2.0|          0.0|
|        893|     3|47.0|    1|    0|    7.0|          1|    0|      1.0|           0.0|          2.0|
|        894|     2|62.0|    0|    0| 9.6875|          0|    1|      0.0|           2.0|          0.0|
|        895|     3|27.0|    0|    0| 8.6625|          0|    1|      0.0|           0.0|          0.0|
|        896|     3|22.0|    1|    1|12.2875|          2|    0|      1.0|           0.0|          2.0|
|        897|     3|14.0|    0|    0|  9.225|          0|    1|      0.0|           0.0|          0.0|
|        898|     3|30.0|    0|    0| 7.6292|          0|    1|      1.0|

In [71]:
df_train.show()

+--------+------+----+-----+-----+-------+-----------+-----+---------+--------------+-------------+
|Survived|Pclass| Age|SibSp|Parch|   Fare|Family_Size|Alone|Sex_index|Embarked_index|Initial_index|
+--------+------+----+-----+-----+-------+-----------+-----+---------+--------------+-------------+
|       0|     3|22.0|    1|    0|   7.25|          1|    0|      0.0|           0.0|          0.0|
|       1|     1|38.0|    1|    0|71.2833|          1|    0|      1.0|           1.0|          2.0|
|       1|     3|26.0|    0|    0|  7.925|          0|    1|      1.0|           0.0|          1.0|
|       1|     1|35.0|    1|    0|   53.1|          1|    0|      1.0|           0.0|          2.0|
|       0|     3|35.0|    0|    0|   8.05|          0|    1|      0.0|           0.0|          0.0|
|       0|     3|33.0|    0|    0| 8.4583|          0|    1|      0.0|           2.0|          0.0|
|       0|     1|54.0|    0|    0|51.8625|          0|    1|      0.0|           0.0|          0.0|


**PREDICTION**

In [72]:
vec_asmbl = VectorAssembler(inputCols=df_train.columns[1:], 
                           outputCol='features')

In [73]:
from pyspark.ml.classification import RandomForestClassifier

In [74]:
rf = RandomForestClassifier(labelCol='Survived', 
                           numTrees=100, maxDepth=3)

In [75]:
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml import Pipeline

pipeline_rf = Pipeline(stages=[vec_asmbl, rf])

paramGrid = ParamGridBuilder().\
            addGrid(rf.maxDepth, [3, 4, 5]).\
            addGrid(rf.minInfoGain, [0., 0.01, 0.1]).\
            addGrid(rf.numTrees, [1000]).\
            build()

selected_model = CrossValidator(estimator=pipeline_rf, 
                                estimatorParamMaps=paramGrid, 
                                evaluator=evaluator, 
                                numFolds=5)

model_final = selected_model.fit(df_train)
pred_train = model_final.transform(df_train)
evaluator.evaluate(pred_train)

22/08/02 09:26:15 WARN DAGScheduler: Broadcasting large task binary with size 1124.2 KiB


22/08/02 09:26:17 WARN DAGScheduler: Broadcasting large task binary with size 2.8 MiB
22/08/02 09:26:19 WARN DAGScheduler: Broadcasting large task binary with size 1086.7 KiB


22/08/02 09:26:21 WARN DAGScheduler: Broadcasting large task binary with size 2.8 MiB
22/08/02 09:26:23 WARN DAGScheduler: Broadcasting large task binary with size 2.2 MiB
22/08/02 09:26:25 WARN DAGScheduler: Broadcasting large task binary with size 1124.2 KiB
22/08/02 09:26:26 WARN DAGScheduler: Broadcasting large task binary with size 2.1 MiB


22/08/02 09:26:29 WARN DAGScheduler: Broadcasting large task binary with size 3.4 MiB
22/08/02 09:26:31 WARN DAGScheduler: Broadcasting large task binary with size 1086.7 KiB


22/08/02 09:26:32 WARN DAGScheduler: Broadcasting large task binary with size 1873.3 KiB


...

22/08/02 09:29:04 WARN DAGScheduler: Broadcasting large task binary with size 2.8 MiB


22/08/02 09:29:06 WARN DAGScheduler: Broadcasting large task binary with size 3.8 MiB
22/08/02 09:29:08 WARN DAGScheduler: Broadcasting large task binary with size 2.2 MiB
22/08/02 09:29:10 WARN DAGScheduler: Broadcasting large task binary with size 1134.7 KiB


22/08/02 09:29:11 WARN DAGScheduler: Broadcasting large task binary with size 2.2 MiB


22/08/02 09:29:14 WARN DAGScheduler: Broadcasting large task binary with size 3.5 MiB


0.835016835016835

In [76]:
pred_test = model_final.transform(df_test)

In [77]:
predictions = pred_test.select('PassengerId', 'prediction')
predictions = predictions.\
                withColumn('Survived', predictions['prediction'].\
                cast('integer')).drop('prediction')
predictions.show(5)

22/08/02 09:29:16 WARN DAGScheduler: Broadcasting large task binary with size 3.4 MiB
+-----------+--------+
|PassengerId|Survived|
+-----------+--------+
|        892|       0|
|        893|       1|
|        894|       0|
|        895|       0|
|        896|       1|
+-----------+--------+
only showing top 5 rows



In [78]:
predictions.show()

22/08/02 09:29:16 WARN DAGScheduler: Broadcasting large task binary with size 3.4 MiB
+-----------+--------+
|PassengerId|Survived|
+-----------+--------+
|        892|       0|
|        893|       1|
|        894|       0|
|        895|       0|
|        896|       1|
|        897|       0|
|        898|       1|
|        899|       0|
|        900|       1|
|        901|       0|
|        902|       0|
|        903|       0|
|        904|       1|
|        905|       0|
|        906|       1|
|        907|       1|
|        908|       0|
|        909|       0|
|        910|       1|
|        911|       1|
+-----------+--------+
only showing top 20 rows



In [79]:
predictions.schema

StructType([StructField('PassengerId', IntegerType(), True), StructField('Survived', IntegerType(), True)])

In [80]:
# Writing csv file in Spark 
predictions.coalesce(1).write.csv('submission_file.csv', header=True)

22/08/02 09:29:17 WARN DAGScheduler: Broadcasting large task binary with size 3.6 MiB
22/08/02 09:29:18 ERROR Utils: Aborting task
org.apache.spark.SparkException: Failed to execute user defined function (VectorAssembler$$Lambda$4083/0x00000008416c9040: (struct<Pclass_double_VectorAssembler_f80f71f1d249:double,Age:double,SibSp_double_VectorAssembler_f80f71f1d249:double,Parch_double_VectorAssembler_f80f71f1d249:double,Fare:double,Family_Size_double_VectorAssembler_f80f71f1d249:double,Alone_double_VectorAssembler_f80f71f1d249:double,Sex_index:double,Embarked_index:double,Initial_index:double>) => struct<type:tinyint,size:int,indices:array<int>,values:array<double>>)
	at org.apache.spark.sql.errors.QueryExecutionErrors$.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala:177)
	at org.apache.spark.sql.errors.QueryExecutionErrors.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCod

Py4JJavaError: An error occurred while calling o16924.csv.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.jobAbortedError(QueryExecutionErrors.scala:638)
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:278)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:186)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:113)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:111)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.executeCollect(commands.scala:125)
	at org.apache.spark.sql.execution.QueryExecution$$anonfun$eagerlyExecuteCommands$1.$anonfun$applyOrElse$1(QueryExecution.scala:98)
	at org.apache.spark.sql.execution.SQLExecution$.$anonfun$withNewExecutionId$6(SQLExecution.scala:109)
	at org.a

In [ ]:
# Reading the saved file from spark 
spark.read.csv('submission_file.csv', header=True).show(4)